In [1]:
#!/usr/bin/env python

!pip3 install fuzzywuzzy

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
data = pd.read_csv("ratings.csv")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


FileNotFoundError: ignored

In [ ]:

#pivot ratings into book features
df_book_features = pd.pivot_table(data=data,
    index='book_id',
    columns='user_id',
    values='rating',fill_value=0)
title_list=df_book_features.index.tolist()
mat_book_features = csr_matrix(df_book_features.values)

In [ ]:
mat_book_features.todense()

matrix([[0, 0, 0, ..., 4, 4, 4],
        [0, 5, 0, ..., 5, 5, 5],
        [0, 0, 0, ..., 0, 0, 4],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn.fit(mat_book_features)
test_title=input("What book have you recently read: ")
query_index=title_list.index(process.extract(test_title,title_list)[0][0])
distances, indices = model_knn.kneighbors(df_book_features.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)
for i in range(0,len(distances.flatten())):
    if i ==0:
        print('Recommendations for {0}:\n'.format(df_book_features.index[query_index]))
    else:
        print('{0}:{1}, with distance of {2}.'.format(i,df_book_features.index[indices.flatten()[i]],distances.flatten()[i]))

model_knn2 = NearestNeighbors(metric='euclidean', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn2.fit(mat_book_features)
distances, indices = model_knn2.kneighbors(df_book_features.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)
for i in range(0,len(distances.flatten())):
    if i ==0:
        print('Recommendations for {0}:\n'.format(df_book_features.index[query_index]))
    else:
        print('{0}:{1}, with distance of {2}.'.format(i,df_book_features.index[indices.flatten()[i]],distances.flatten()[i]))


What book have you recently read: 1116
Recommendations for 1116:

1:1443, with distance of 0.12686222762730215.
2:2171, with distance of 0.26863786028416303.
3:1085, with distance of 0.45111236311562164.
4:1125, with distance of 0.4754427537605015.
5:1409, with distance of 0.5174833835954786.
Recommendations for 1116:

1:1443, with distance of 37.36308338453881.
2:2171, with distance of 52.29722745997153.
3:4809, with distance of 67.446274915669.
4:1409, with distance of 67.68308503607086.
5:1125, with distance of 67.88961628997471.
